<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ShopRunner/collie/blob/main/tutorials/03_advanced_matrix_factorization.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ShopRunner/collie/blob/main/tutorials/03_advanced_matrix_factorization.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/ShopRunner/collie/main/tutorials/03_advanced_matrix_factorization.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

In [ ]:
# for Collab notebooks, we will start by installing the ``collie`` library
!pip install collie --quiet

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%env DATA_PATH data/

env: DATA_PATH=data/


In [ ]:
import os

import joblib
from pytorch_lightning.utilities.seed import seed_everything

from collie.interactions import ApproximateNegativeSamplingInteractionsDataLoader
from collie.metrics import auc, evaluate_in_batches, mapk, mrr
from collie.model import CollieTrainer, MatrixFactorizationModel
from collie.movielens import read_movielens_df_item

## Load Data From ``01_prepare_data`` Notebook 
If you're running this locally on Jupyter, you should be able to run the next cell quickly without a problem! If you are running this on Colab, you'll need to regenerate the data by running the cell below that, which should only take a few extra seconds to complete. 

In [ ]:
try:
    # let's grab the ``Interactions`` objects we saved in the last notebook
    train_interactions = joblib.load(os.path.join(os.environ.get('DATA_PATH', 'data/'),
                                                  'train_interactions.pkl'))
    val_interactions = joblib.load(os.path.join(os.environ.get('DATA_PATH', 'data/'),
                                                'val_interactions.pkl'))
except FileNotFoundError:
    # we're running this notebook on Colab where results from the first notebook are not saved
    # regenerate this data below
    from collie.cross_validation import stratified_split
    from collie.interactions import Interactions
    from collie.movielens import read_movielens_df
    from collie.utils import convert_to_implicit, remove_users_with_fewer_than_n_interactions


    df = read_movielens_df(decrement_ids=True)
    implicit_df = convert_to_implicit(df, min_rating_to_keep=4)
    implicit_df = remove_users_with_fewer_than_n_interactions(implicit_df, min_num_of_interactions=3)

    interactions = Interactions(
        users=implicit_df['user_id'],
        items=implicit_df['item_id'],
        ratings=implicit_df['rating'],
        allow_missing_ids=True,
    )

    train_interactions, val_interactions = stratified_split(interactions, test_p=0.1, seed=42)


print('Train:', train_interactions)
print('Val:  ', val_interactions)

Train: Interactions object with 49426 interactions between 943 users and 1674 items, returning 10 negative samples per interaction.
Val:   Interactions object with 5949 interactions between 943 users and 1674 items, returning 10 negative samples per interaction.


# More Advanced Ways to Train a Matrix Factorization Model 
Thus far in the tutorials, we have only walked through the Collie necessities: preparing data, data splits, training a model, and evaluating a model. 

This optional notebook will be an extension of the last notebook, diving into some of the more advanced features of Collie we can use without the use of metadata. 

In [ ]:
# this handy PyTorch Lightning function fixes random seeds across all the libraries used here
seed_everything(22)

Global seed set to 22


22

## Faster Data Loading Through Approximate Negative Sampling 

With sufficiently large enough data, verifying that each negative sample is one a user has *not* interacted with becomes expensive. With many items, this can soon become a bottleneck in the training process. 

Yet, when we have many items, the chances a user has interacted with most is increasingly rare. Say we have ``1,000,000`` items and we want to sample ``10`` negative items for a user that has positively interacted with ``200`` items. The chance that we accidentally select a positive item in a random sample of ``10`` items is just ``0.2%``. At that point, it might be worth it to forgo the expensive check to assert our negative sample is true, and instead just randomly sample negative items with the hope that most of the time, they will happen to be negative. 

This is the theory behind the ``ApproximateNegativeSamplingInteractionsDataLoader``, an alternate DataLoader built into Collie. Let's train a model with this below, noting how similar this procedure looks to that in the previous tutorial. 

In [ ]:
train_loader = ApproximateNegativeSamplingInteractionsDataLoader(train_interactions, batch_size=1024, shuffle=True)
val_loader = ApproximateNegativeSamplingInteractionsDataLoader(val_interactions, batch_size=1024, shuffle=False)

In [ ]:
model = MatrixFactorizationModel(
    train=train_loader,
    val=val_loader,
    embedding_dim=10,
    lr=1e-2,
)

In [ ]:
trainer = CollieTrainer(model, max_epochs=10, deterministic=True)

trainer.fit(model)
model.eval()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type            | Params
----------------------------------------------------
0 | user_biases     | ZeroEmbedding   | 943   
1 | item_biases     | ZeroEmbedding   | 1.7 K 
2 | user_embeddings | ScaledEmbedding | 9.4 K 
3 | item_embeddings | ScaledEmbedding | 16.7 K
4 | dropout         | Dropout         | 0     
----------------------------------------------------
28.8 K    Trainable params
0         Non-trainable params
28.8 K    Total params
0.115     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch     7: reducing learning rate of group 0 to 1.0000e-03.
Epoch     7: reducing learning rate of group 0 to 1.0000e-03.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

MatrixFactorizationModel(
  (user_biases): ZeroEmbedding(943, 1)
  (item_biases): ZeroEmbedding(1674, 1)
  (user_embeddings): ScaledEmbedding(943, 10)
  (item_embeddings): ScaledEmbedding(1674, 10)
  (dropout): Dropout(p=0.0, inplace=False)
)

In [ ]:
mapk_score, mrr_score, auc_score = evaluate_in_batches([mapk, mrr, auc], val_interactions, model)

print(f'MAP@10 Score: {mapk_score}')
print(f'MRR Score:    {mrr_score}')
print(f'AUC Score:    {auc_score}')

  0%|          | 0/48 [00:00<?, ?it/s]

MAP@10 Score: 0.05060812609426882
MRR Score:    0.1743359738158066
AUC Score:    0.9044078140501763


We're seeing a small hit on performance and only a marginal improvement in training time compared to the standard ``MatrixFactorizationModel`` we trained in Tutorial ``02`` because MovieLens 100K has so few items. ``ApproximateNegativeSamplingInteractionsDataLoader`` is especially recommended for when we have more items in our data and training times need to be optimized. 

For more details on this and other DataLoaders in Collie (including those for out-of-memory datasets), check out the [docs](https://collie.readthedocs.io/en/latest/index.html)! 

## Multiple Optimizers 

Training recommendation models at ShopRunner, we have encountered something we call "the curse of popularity." 

This is best thought of in the viewpoint of a model optimizer - say we have a user, a positive item, and several negative items that we hope have recommendation scores that score lower than the positive item. As an optimizer, you can either optimize every single embedding dimension (hundreds of parameters) to achieve this, or instead choose to score a quick win by optimizing the bias terms for the items (just add a positive constant to the positive item and a negative constant to each negative item). 

While we clearly want to have varied embedding layers that reflect each user and item's taste profiles, some models learn to settle for popularity as a recommendation score proxy by over-optimizing the bias terms, essentially just returning the same set of recommendations for every user. Worst of all, since popular items are... well, popular, **the loss of this model will actually be decent, solidifying the model getting stuck in a local loss minima**. 

To counteract this, Collie supports multiple optimizers in a ``MatrixFactorizationModel``. With this, we can have a faster optimizer work to optimize the embedding layers for users and items, and a slower optimizer work to optimize the bias terms. With this, we impel the model to do the work actually coming up with varied, personalized recommendations for users while still taking into account the necessity of the bias (popularity) terms on recommendations. 

At ShopRunner, we have seen significantly better metrics and results from this type of model. With Collie, this is simple to do, as shown below. 

In [ ]:
model = MatrixFactorizationModel(
    train=train_interactions,
    val=val_interactions,
    embedding_dim=10,
    lr=1e-2,
    bias_lr=1e-1,
    optimizer='adam',
    bias_optimizer='sgd',
)

In [ ]:
trainer = CollieTrainer(model, max_epochs=10, deterministic=True)

trainer.fit(model)
model.eval()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type            | Params
----------------------------------------------------
0 | user_biases     | ZeroEmbedding   | 943   
1 | item_biases     | ZeroEmbedding   | 1.7 K 
2 | user_embeddings | ScaledEmbedding | 9.4 K 
3 | item_embeddings | ScaledEmbedding | 16.7 K
4 | dropout         | Dropout         | 0     
----------------------------------------------------
28.8 K    Trainable params
0         Non-trainable params
28.8 K    Total params
0.115     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch     7: reducing learning rate of group 0 to 1.0000e-03.
Epoch     7: reducing learning rate of group 0 to 1.0000e-02.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

MatrixFactorizationModel(
  (user_biases): ZeroEmbedding(943, 1)
  (item_biases): ZeroEmbedding(1674, 1)
  (user_embeddings): ScaledEmbedding(943, 10)
  (item_embeddings): ScaledEmbedding(1674, 10)
  (dropout): Dropout(p=0.0, inplace=False)
)

In [ ]:
mapk_score, mrr_score, auc_score = evaluate_in_batches([mapk, mrr, auc], val_interactions, model)

print(f'MAP@10 Score: {mapk_score}')
print(f'MRR Score:    {mrr_score}')
print(f'AUC Score:    {auc_score}')

  0%|          | 0/48 [00:00<?, ?it/s]

MAP@10 Score: 0.049999912405002886
MRR Score:    0.17664332823895093
AUC Score:    0.9056327464474234


Again, we're not seeing as much performance increase here compared to the model in Tutorial ``02`` because MovieLens 100K has so few items. For a more dramatic difference, try training this model on a larger dataset, such as MovieLens 10M, adjusting the architecture-specific hyperparameters, or train longer. 

## Item-Item Similarity 

While we've trained every model thus far to work for member-item recommendations (given a *member*, recommend *items* - think of this best as "Personalized recommendations for you"), we also have access to item-item recommendations for free (given a seed *item*, recommend similar *items* - think of this more like "People who interacted with this item also interacted with..."). 

With Collie, accessing this is simple! 

In [ ]:
# start by reading in the item information with MovieLens data
df_item = read_movielens_df_item()

# all IDs must start at 0 in Collie, so we'll make that adjustment here
df_item['item_id'] -= 1

In [ ]:
df_item.loc[df_item['movie_title'] == 'Star Wars (1977)']

,item_id,movie_title,release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
49,49,Star Wars (1977),1977-01-01,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,0,1,1,0,1,0


In [ ]:
# let's start by finding movies similar to Star Wars (1977)
item_similarities = model.item_item_similarity(item_id=49)


item_similarities

49      1.000000
180     0.933118
0       0.895945
256     0.785682
126     0.775992
          ...   
1507   -0.947865
1422   -0.953469
906    -0.954468
1195   -0.959211
1357   -0.973927
Length: 1674, dtype: float64

In [ ]:
df_item.iloc[item_similarities.index][:5]

,item_id,movie_title,release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
49,49,Star Wars (1977),1977-01-01,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,0,1,1,0,1,0
180,180,Return of the Jedi (1983),1997-03-14,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,0,0,0,...,0,0,0,0,0,1,1,0,1,0
0,0,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
256,256,Men in Black (1997),1997-07-04,http://us.imdb.com/M/title-exact?Men+in+Black+...,0,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
126,126,"Godfather, The (1972)",1972-01-01,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


These films seem similar to Star Wars <sup>1</sup>! 

Best of all, the ``item_item_similarity`` method is available in all Collie models, not just ``MatrixFactorizationModel``! 

In the next two tutorials, we will incorporate item metadata into recommendations for even better results. 

<font size="1"><sup>1</sup> <span id="fn1">Note that the author of this tutorial has never seen Star Wars, and thus cannot accurately say that these films are similar. Sorry to Star Wars fans.</span></font>

----- 